In [3]:
import pandas as pd
import numpy as np

# Configuração de número de registros e seed para reprodutibilidade
np.random.seed(42)
num_funcionarios = 50
num_periodos = 12
num_regioes = 5

# Dimensão Funcionários
funcionarios = pd.DataFrame({
    "funcionario_id": range(1, num_funcionarios + 1),
    "nome": [f"Funcionario_{i}" for i in range(1, num_funcionarios + 1)],
    "departamento": np.random.choice(["Vendas", "TI", "Marketing", "RH"], num_funcionarios),
    "cargo": np.random.choice(["Analista", "Supervisor", "Coordenador"], num_funcionarios)
})

# Dimensão Períodos
periodos = pd.DataFrame({
    "periodo_id": range(1, num_periodos + 1),
    "mes": [f"{i:02d}" for i in range(1, 13)],
    "ano": 2024
})

# Dimensão Regiões
regioes = pd.DataFrame({
    "regiao_id": range(1, num_regioes + 1),
    "nome_regiao": [f"Regiao_{i}" for i in range(1, num_regioes + 1)]
})

# Tabela Fato Performance
performance = pd.DataFrame({
    "funcionario_id": np.random.choice(funcionarios["funcionario_id"], num_funcionarios * num_periodos),
    "periodo_id": np.random.choice(periodos["periodo_id"], num_funcionarios * num_periodos),
    "regiao_id": np.random.choice(regioes["regiao_id"], num_funcionarios * num_periodos),
    "meta_definida": np.random.randint(50, 101, num_funcionarios * num_periodos),
    "meta_alcancada": np.random.randint(0, 101, num_funcionarios * num_periodos)
})

# Garantir unicidade por funcionário e período
performance = performance.groupby(
    ["funcionario_id", "periodo_id"], as_index=False
).agg({
    "meta_definida": "mean",  # Média das metas definidas
    "meta_alcancada": "mean",  # Média das metas alcançadas
    "regiao_id": "first"  # Seleciona uma única região arbitrariamente
})

# Calcular Percentual de Alcance e ICP novamente
performance["percentual_alcance"] = (performance["meta_alcancada"] / performance["meta_definida"] * 100).round(2)
performance["ICP"] = performance["percentual_alcance"].apply(lambda x: 1000 if x >= 100 else (500 if x >= 80 else 0))

# Adicionar coluna para contar períodos seguidos abaixo de 80% de alcance
performance = performance.sort_values(by=["funcionario_id", "periodo_id"])

def contar_periodos_abaixo_80(grupo):
    contador = 0
    max_contador = 0
    contagens = []
    for percentual in grupo["percentual_alcance"]:
        if percentual < 80:
            contador += 1
        else:
            max_contador = max(max_contador, contador)
            contador = 0
        contagens.append(contador)
    grupo["periodos_abaixo_80"] = contagens
    return grupo

performance = performance.groupby("funcionario_id").apply(contar_periodos_abaixo_80).reset_index(drop=True)

# Amostras dos Dados
print("Funcionários:")
print(funcionarios.head())
funcionarios.to_excel('funcionarios.xlsx', index=False)

print("\nPeríodos:")
print(periodos.head())
periodos.to_excel('periodos.xlsx', index=False)

print("\nRegiões:")
print(regioes.head())
regioes.to_excel('regioes.xlsx', index=False)

print("\nPerformance:")
print(performance.head())
performance.to_excel('performance.xlsx', index=False)


C:\Users\Bruno\AppData\Local\Temp\ipykernel_4180\3482787232.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  performance = performance.groupby("funcionario_id").apply(contar_periodos_abaixo_80).reset_index(drop=True)


Funcionários:
   funcionario_id           nome departamento        cargo
0               1  Funcionario_1    Marketing  Coordenador
1               2  Funcionario_2           RH   Supervisor
2               3  Funcionario_3       Vendas  Coordenador
3               4  Funcionario_4    Marketing  Coordenador
4               5  Funcionario_5    Marketing     Analista

Períodos:
   periodo_id mes   ano
0           1  01  2024
1           2  02  2024
2           3  03  2024
3           4  04  2024
4           5  05  2024

Regiões:
   regiao_id nome_regiao
0          1    Regiao_1
1          2    Regiao_2
2          3    Regiao_3
3          4    Regiao_4
4          5    Regiao_5

Performance:
   funcionario_id  periodo_id  meta_definida  meta_alcancada  regiao_id  \
0               1           1      75.666667       70.333333          2   
1               1           2      93.000000       41.000000          4   
2               1           3      75.000000       81.000000          1   
3  